In [ ]:
import sys
sys.path.insert(0, "../..")
sys.path.insert(1, '../')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from scipy.special import erf, erfc, jv, yv,exp1
from tqdm.notebook import tqdm
from datetime import timedelta, datetime
from shapely.geometry import Polygon, Point
import itertools 
import swifter
from fgem.utils.geocluster_utils import TEA
tqdm.pandas()

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import StrMethodFormatter
import geopandas as gpd


prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from fgem.subsurface import *
from fgem.powerplant import *
from fgem.utils.utils import *
from fgem.utils.config import *
from fgem.utils.constants import *
from fgem.world import World
from datetime import timedelta

# reload
from tqdm.notebook import tqdm

In [ ]:
def constant_strategy(project, mass_flow=100):
    """Constant change producer mass flow rates"""
    m_prd = np.array(project.num_prd*[mass_flow]).astype(float)
    m_inj = np.array(project.num_inj*[m_prd.sum()/project.num_inj]).astype(float)
    return m_prd, m_inj

def maximal_power_generation_strategy(project, max_mass_flow=200):
    """Control wells to maintain a constant power plant output"""
    power_output_MWh_kg = project.pp.power_output_MWh_kg# project.pp.compute_geofluid_consumption(project.reservoir.T_prd_wh.mean(), project.state.T0)
    required_mass_flow_per_well = project.ppc / (power_output_MWh_kg * 3600 * project.num_prd + SMALL_NUM)

    m_prd = np.minimum(max_mass_flow, np.array(project.num_prd*[required_mass_flow_per_well])).astype(float)
    m_inj = np.array(project.num_inj*[m_prd.sum()/project.num_inj]).astype(float)

    return m_prd, m_inj

def prepare_case(depth, tres, surface_temp, num_prd, m_well, lateral_spacing,
                 prd_well_diam, inj_well_diam, lateral_diam, numberoflaterals, lateral_length,
                 well_depth, L, ppc=None, reservoir_simulator_settings={"on": False, "accuracy": 5}, T_inj=60,
                 closedloop_design="Default", dx=None):
    columns = ["Depth", "Tres", "Geothermal_Gradient", "Installed_PPC", "No. Producers"]
    records = []
    geothermal_gradient = (tres-surface_temp)/depth*1000 #degC/km 
        
    half_lateral_length = lateral_length/2
    dx = dx if dx else lateral_length//5

    times_arr = np.linspace(0, L * 8760, L)
    
    reservoir = ULoopSBT(Tres_init=tres, surface_temp=surface_temp, geothermal_gradient=geothermal_gradient, 
                     prd_well_diam=prd_well_diam, inj_well_diam=inj_well_diam, lateral_diam=lateral_diam, 
                     numberoflaterals=numberoflaterals, well_depth = depth, lateral_spacing=lateral_spacing,
                     half_lateral_length = half_lateral_length, Pres_init=100, L=L, 
                     time_init=datetime(year=2024, month=1, day=1), num_prd=1, num_inj=1,
                     waterloss=0.0, power_plant_type="Binary", pumpeff=0.75, times_arr=times_arr, reservoir_simulator_settings=reservoir_simulator_settings,
                     closedloop_design=closedloop_design, 
                    )
    
    temps = []
    for i in range(len(times_arr)-1):
        reservoir.timestep = timedelta(hours=float(times_arr[i+1] - times_arr[i]))
        reservoir.step(np.array(m_well), np.array(m_well), T_inj, surface_temp)
        temps.append(reservoir.T_prd_wh.mean())

    pp = ORCPowerPlant(ppc=ppc, Tres=np.mean(temps))
    gc = pp.compute_geofluid_consumption(np.mean(temps), surface_temp) #MWh/kg
    m_g = num_prd * m_well #kg/s
    ppc_design = nonzero(gc) * 3600 * m_g
    ppc = ppc if ppc else ppc_design
    print(f"PPC Design: ", ppc_design)

#     print(f"Temperature Decline: {temps} (avg: {np.mean(temps):0.1f})")
    print(f"Plateau Avg Production Temperature : {np.mean(temps[-10:]):0.1f}")
    print(f"Avg Production Temperature : {np.mean(temps):0.1f}")
    print(f"Upstream Capacity: {ppc_design:0.2f} \n")
    records.append([depth, tres, geothermal_gradient, ppc, num_prd])

    df = pd.DataFrame(records, columns=columns)
    return df

def prepare_world(depth, state, tres, lateral_spacing, lateral_length, numberoflaterals, num_prd, well_diam, 
                  m_well, drilling_cost=1000, L=30, ppc=None, reservoir_simulator_settings={"on": False, "accuracy": 5}, T_inj=60,
                  resample=None, dx=None, closedloop_design="Default"):

    df = prepare_case(depth, tres, surface_temp, num_prd, m_well,lateral_spacing,
                      well_diam, well_diam, well_diam, numberoflaterals, lateral_length,
                      depth, L, ppc, reservoir_simulator_settings, T_inj, closedloop_design, dx)
    
    ### Configure Case ###
    record = df[df["Depth"] == depth].iloc[0]
    print(record)
    
    config_filename = f"config.json"
    config = get_config_from_json(config_filename)
#     config["metadata"]["market_dir"] = f"{project_name}_{cambium_scenario}"
    config["metadata"]["state"] = state
    config["economics"]["L"] = L
    config["economics"]["drilling_cost"] = drilling_cost
    config["upstream"]['inj_prd_ratio'] = 1
    config["downstream"]["powerplant"]["ppc"] = record["Installed_PPC"]
    config["downstream"]["powerplant"]["bypass"] = False
    config["upstream"]["surface_temp"] = surface_temp
    config["upstream"]["Tres_init"] = tres
    config["upstream"]["well_depth"] = depth
    config["upstream"]["prd_well_diam"] = well_diam
    config["upstream"]["inj_well_diam"] = well_diam
    config["upstream"]["lateral_diam"] = well_diam
    config["upstream"]["geothermal_gradient"] = record["Geothermal_Gradient"]
    config["upstream"]["num_prd"] = int(record["No. Producers"])
    config["upstream"]["num_inj"] = int(record["No. Producers"] * config["upstream"]['inj_prd_ratio']) #int(record["No. Producers"])
    config["upstream"]["lateral_length"] = lateral_length # meters
    config["upstream"]["V_res"] = V_res #SRV * int(record["No. Producers"])
    config["upstream"]["reservoir_type"] = "uloop"
    config["upstream"]["numberoflaterals"] = numberoflaterals
    config["upstream"]["waterloss"] = 0.05
    config["upstream"]["DSR"] = 1.0
    config["upstream"]["lateral_spacing"] = lateral_spacing
    config["upstream"]["PumpingModel"] = "ClosedLoop"
    config["upstream"]["closedloop_design"] = closedloop_design
    config["metadata"]["reservoir_simulator_settings"] = reservoir_simulator_settings
    config["market"]["resample"] = resample
    config["upstream"]["dx"] = dx
    
    project_initialized = World(config)
    
    return project_initialized, config

### ULoopSBT Timestepping Tests

In [ ]:
# logspace
T_amb = 20
L = 10
wellradius = 0.0762
numberoflaterals = 1
lateralflowallocation = numberoflaterals * [1/numberoflaterals]
lateral_length = 2000
half_lateral_length = lateral_length/2
dx = lateral_length//5 # keep both well_depth and lateral_length as multiples of 1000 for things to work properly and quickly
reservoir_simulator_settings = {"on": False, "accuracy": 1, "DynamicFluidProperties": True}


scenarios = {
               "SBT Linear (100 steps)": {"times_arr": np.linspace(0, L * 8760, 100)},
             "SBT Linear (1000 steps)": {"times_arr": np.linspace(0, L * 8760, 1000)},
              "SBT Log (100 steps)": {"times_arr": np.logspace(0, np.log10(L * 8760), 100)},
             "SBT Log (1000 steps)": {"times_arr": np.logspace(0, np.log10(L * 8760), 1000)},
            }

for name, scenario in tqdm(scenarios.items()):
    times_arr = scenario["times_arr"]

    reservoir = ULoopSBT(Tres_init=60, cp_f=4200, rho_f=1000, k_f=0.68, mu_f=600*10**-6, surface_temp=T_amb,
                         geothermal_gradient=60, k_m = 2.83, c_m = 825, rho_m=2875,
                         prd_well_diam=2*wellradius, inj_well_diam=2*wellradius, lateral_diam=2*wellradius, 
                         numberoflaterals=numberoflaterals, lateralflowmultiplier=1, fullyimplicit=1.,
                         well_depth = 2000, half_lateral_length = half_lateral_length,
                         Pres_init=100, L=L, time_init=datetime(year=2024, month=1, day=1), num_prd=1, num_inj=1,
                         waterloss=0.05, power_plant_type="Binary", pumpeff=0.75, times_arr=times_arr, dx=dx,
                         reservoir_simulator_settings=reservoir_simulator_settings, closedloop_design="Default", ramey=False
                        )
#     fig = reservoir.plot_laterals(dpi=100)
    m_prd = np.array([40])
    T_inj = 20
    T_bh = [reservoir.Tres]
    T_injs = [T_inj]
    for i in tqdm(range(len(times_arr)-1)):
        reservoir.timestep = timedelta(hours=float(times_arr[i+1] - times_arr[i]))
        reservoir.step(m_prd=m_prd, 
                       T_inj=T_inj, 
                       T_amb=T_amb)
        T_bh.append(reservoir.Tres)
        T_injs.append(T_inj)
    
    T_bh = np.array(T_bh)
    T_injs = np.array(T_injs)

    scenarios[name]["T_bh"] = T_bh
    scenarios[name]["T_injs"] = T_injs

In [ ]:
# Roland's Numbers
roland_times = np.array([1, 7, 30, 365, 730, 3650]) * 24
roland_T_bh = np.array([40.11247583, 35.34166404, 33.05493187, 30.01457369, 29.40924561, 28.25086947])
fig, ax = plt.subplots(1, 1, figsize=(8,4))
plt.plot(roland_times/(8760), roland_T_bh, 'black', label="Ramey's Analytical Solution")

for name, scenario in scenarios.items():
    mask = (scenario["times_arr"]>72) #skip first jump
    plt.plot(scenario["times_arr"][mask]/(8760),
             scenario["T_bh"][mask], 
             label=name)

plt.ylabel("Production Wellhead Temperature [$^\circ C$]")
plt.xlabel("Years")
plt.legend(loc="upper right")

### Eavor 2.0 Validation

In [ ]:
# logspace
T_inj = 60
surface_temp = 10 #deg C
num_prd = 1
m_well = 80 #kg/s
well_diam = 0.216#0.1524
numberoflaterals = 12
lateral_length = 6500 # length from prd toe to inj toe
well_depth = 7500 #18000/3.28 # meters
L = 30
drilling_cost=1000 #$/meter
half_lateral_length = lateral_length/2
ppc = 20
state = "NM"
geothermal_gradient = 60 #(250-surface_temp)/(18000/3.28) #deg C/m based on Eavor publications
tres = surface_temp + geothermal_gradient * well_depth/1000
lateral_spacing = 30 #meters
dx = None
reservoir_simulator_settings = {"on": False, "accuracy": 5, "DynamicFluidProperties": True}

scenarios = {
              "$\dot{m}$: 40 kg/s & $T_{inj}$: 20°C (Baseline)": {
                  "strategy": lambda reservoir: (np.array([m_well]), T_inj)},
            }

for name, scenario in tqdm(scenarios.items()):
    times_arr = np.linspace(0, L * 8760, 200)
#     times_arr = np.concatenate((np.linspace(0,9900,100), np.logspace(np.log10(100*100), np.log10(L*365*24*3600), 75)))/3600
#     times_arr = np.logspace(0, np.log10(L * 8760), 200)
    
    reservoir = ULoopSBT(Tres_init=tres, cp_f=4200, rho_f=1000, k_f=0.68, mu_f=600*10**-6, surface_temp=surface_temp,
                         geothermal_gradient=geothermal_gradient, k_m = 2.5, c_m = 1112, rho_m=2663,
                         prd_well_diam=well_diam, inj_well_diam=well_diam, lateral_diam=well_diam, 
                         numberoflaterals=numberoflaterals, lateralflowmultiplier=1, fullyimplicit=1., reservoir_simulator_settings=reservoir_simulator_settings,
                         well_depth = well_depth, half_lateral_length = half_lateral_length,
                         Pres_init=100, L=L, time_init=datetime(year=2024, month=1, day=1), num_prd=num_prd, num_inj=num_prd,
                         waterloss=0, power_plant_type="Binary", pumpeff=0.75, times_arr=times_arr, lateral_spacing=lateral_spacing,
                         dx=dx, ramey=False
                        )
    
    T_bh = [reservoir.Tres]
    T_injs = [T_inj]
    for i in tqdm(range(len(times_arr)-1)):
        m_prd, T_inj = scenario["strategy"](reservoir)
        
        reservoir.timestep = timedelta(hours=float(times_arr[i+1] - times_arr[i]))
        reservoir.step(m_prd=m_prd,  
                       T_inj=T_inj,
                       T_amb=surface_temp)
        T_bh.append(reservoir.Tres)
        T_injs.append(T_inj)

    T_bh = np.array(T_bh)
    T_injs = np.array(T_injs)

    scenarios[name]["T_bh"] = T_bh
    scenarios[name]["T_injs"] = T_injs

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4))

for name, scenario in scenarios.items():
    mask = (times_arr>=0) #skip first jump
    plt.plot(times_arr[mask]/(8760),
             scenario["T_bh"][mask],
             label=name)

plt.ylabel("Production Wellhead Temperature [$^\circ C$]")
plt.xlabel("Years")
# plt.xscale("log")
# plt.legend(loc=(1.1,0.5))

### Variable Flow Rate Tests

In [ ]:
# logspace
T_amb = 20
L = 10
wellradius = 0.0762
numberoflaterals = 3
lateral_length = 2000
half_lateral_length = lateral_length/2
dx = lateral_length//5
lateralflowallocation = numberoflaterals * [1/numberoflaterals]
reservoir_simulator_settings = {"on": False, "accuracy": 5, "DynamicFluidProperties": True}

def seasonal_m(reservoir):
    if (reservoir.time_curr.month>=6) & (reservoir.time_curr.month>=9):
        return np.array([80]), 20
    else:
        return np.array([40]), 20

def seasonal_Tinj(reservoir):
    if (reservoir.time_curr.month>=6) & (reservoir.time_curr.month>=9):
        return np.array([40]), 40
    else:
        return np.array([40]), 20
    
def seasonal_m_Tinj(reservoir):
    if (reservoir.time_curr.month>=6) & (reservoir.time_curr.month>=9):
        return np.array([80]), 40
    else:
        return np.array([40]), 20
    
scenarios = {
              "$\dot{m}$: 40 kg/s & $T_{inj}$: 20°C (Baseline)": {"strategy": lambda reservoir: (np.array([40]), 20)},
              "$\dot{m}$: 80 kg/s & $T_{inj}$: 20°C": {"strategy": lambda reservoir: (np.array([80]), 20)},
              "$\dot{m}$: seasonal & $T_{inj}$: 20°C": {"strategy": seasonal_m},
              "$\dot{m}$: 40 kg/s & $T_{inj}$: seasonal": {"strategy": seasonal_Tinj},
              "$\dot{m}$: seasonal & $T_{inj}$: seasonal": {"strategy": seasonal_m_Tinj},
    
            }

for name, scenario in tqdm(scenarios.items()):
    times_arr = np.linspace(0, L * 8760, 100)

    reservoir = ULoopSBT(Tres_init=20, cp_f=4200, rho_f=1000, k_f=0.68, mu_f=600*10**-6, surface_temp=T_amb,
                         geothermal_gradient=60, k_m = 2.83, c_m = 825, rho_m=2875,
                         prd_well_diam=2*wellradius, inj_well_diam=2*wellradius, lateral_diam=2*wellradius, 
                         numberoflaterals=numberoflaterals, lateralflowmultiplier=1, fullyimplicit=1.,
                         well_depth = 2000, half_lateral_length = half_lateral_length,
                         Pres_init=100, L=L, time_init=datetime(year=2024, month=1, day=1), num_prd=1, num_inj=1,
                         waterloss=0.05, power_plant_type="Binary", pumpeff=0.75, times_arr=times_arr, dx=dx,
                         reservoir_simulator_settings=reservoir_simulator_settings
                        )

    m_prd = np.array([40])
    m_inj = np.array([40])
    T_inj = 20
    T_bh = [reservoir.Tres]
    T_injs = [T_inj]
    for i in tqdm(range(len(times_arr)-1)):
        m_prd, T_inj = scenario["strategy"](reservoir)
        m_inj = m_prd
        
        reservoir.timestep = timedelta(hours=float(times_arr[i+1] - times_arr[i]))
        reservoir.step(m_prd=m_prd, 
                       T_inj=T_inj, 
                       T_amb=T_amb)
        T_bh.append(reservoir.Tres)
        T_injs.append(T_inj)
    
    T_bh = np.array(T_bh)
    T_injs = np.array(T_injs)

    scenarios[name]["T_bh"] = T_bh
    scenarios[name]["T_injs"] = T_injs

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4))

for name, scenario in scenarios.items():
    mask = (times_arr>72) #skip first jump
    plt.plot(times_arr[mask]/(8760),
             scenario["T_bh"][mask], 
             label=name)

plt.ylabel("Production Wellhead Temperature [$^\circ C$]")
plt.xlabel("Years")
plt.legend(loc=(1.1,0.5))

### Repeat Tests with FGEM directly

In [ ]:
# further processing
cambium_scenario = "MidCase"
project_name = "CEE219"
V_res = 18 #km2

In [ ]:
# df = pd.read_csv("../../Data/AL_Cambium22_HighRECost_0_0_0.0/Weather.csv")
# df["Date"] = pd.to_datetime(df.time if "time" in df.columns else df.Date)
# df = df.resample(rule=resample, on='Date').last().reset_index()

In [ ]:
# Eavor Case:
# location, depth, Tres: https://www.eavor.com/press-releases/success-at-eavors-new-mexico-project-triggers-follow-on-strategic-investments/
# case study 20 MW, production profiles: https://www.eavor.com/what-the-experts-say/eavor-loop-dispatchability-case-study-for-ppa-with-utility/
T_inj = 92#60
surface_temp = 18 #deg C
num_prd = 1#2#1
m_well = 100#75#80 #kg/s
well_diam = 0.216#0.1524
numberoflaterals = 12#4#12
lateral_length = 5000#20000#6500 # length from prd toe to inj toe
well_depth = 7000#7500 #18000/3.28 # meters
L = 5#10 #30
drilling_cost=1000 #$/meter
half_lateral_length = lateral_length/2
ppc = 15#500
state = "NM"
geothermal_gradient = 60 #(250-surface_temp)/(18000/3.28) #deg C/m based on Eavor publications
tres = surface_temp + geothermal_gradient * well_depth/1000
lateral_spacing = 30 #meters
dx = 500
reservoir_simulator_settings = {"on": False, "accuracy": 1, "DynamicFluidProperties": True}
resample = "1d"


In [ ]:
strategy = "baseload"#"flexible"

project_initialized, config = prepare_world(well_depth, state, tres, lateral_spacing, lateral_length, numberoflaterals, num_prd, well_diam, 
                                m_well, drilling_cost, L, ppc, reservoir_simulator_settings, T_inj, resample, dx)

In [ ]:
strategy = "baseload"#"flexible"

project_initialized, config = prepare_world(well_depth, state, tres, lateral_spacing, lateral_length, numberoflaterals, num_prd, well_diam, 
                                m_well, drilling_cost, L, ppc, reservoir_simulator_settings, T_inj, resample, dx)
m_max = m_well * 1.5
project = deepcopy(project_initialized)
# big_step = 100 #24 * 10 # 10 days
for i in tqdm(range(project.max_simulation_steps-1)):
# for i in tqdm(range(1000)):
    days_passed = (project.time_curr - project.time_init).days
#     hours_passed = days_passed * 24
#     project.timestep = timedelta(hours=big_step) if hours_passed < 8760 * 30 else timedelta(hours=1)
    if strategy == "baseload":
        m_prd, m_inj = constant_strategy(project, mass_flow=m_well)
    else:
        m_prd, m_inj = maximal_power_generation_strategy(project, max_mass_flow=m_max)
    project.step_update_record(m_prd=m_prd, m_inj=m_inj)
        
    if  i % 365 == 0:
        print(f"Year-{days_passed/365:0.0f}: {project.reservoir.Tres} deg C {project.effective_ppc} MW   {project.m_bypass} kg/s")
project.compute_economics()
NPV, ROI, PBP, IRR, PPA_NPV, PPA_ROI, PPA_PBP, PPA_IRR, LCOE, NET_GEN, df_annual = compute_npv(project.df_records, project.capex_total, project.opex_total,
                                                          project.baseline_year, project.L, project.d, ppa_price=75)

# print economics
print(f"LCOE: {LCOE:.0f} $/MWh")
print(f"NPV: {NPV:.0f} $MM")
print(f"ROI: {ROI:.1f} %")
print(f"PBP: {PBP:.0f} yrs")

# pickle.dump(projects, open("projects.pkl", "wb"))

In [ ]:
# pickle.dump(project, open("temp_project.pkl", "wb"))
# project = pickle.load(open("temp_project.pkl", "rb"))

In [ ]:
qdict = {
#             "LMP [$/MWh]": "Electricity Price \n [$/MWh]",
          "Atm Temp [deg C]": "Ambient Temp. \n [$\degree C$]",
          "Res Temp [deg C]": "Reservoir Temp. \n [$\degree C$]",
          'Inj Temp [deg C]': "Intector Temp. \n [$\degree C$]",
          "Net Power Output [MWe]": "Net Generation \n [MWh]",
          'M_Produced [kg/s]': "Field Production \n [kg/s]",
          "Pumping Power [MWe]": "Pumping Power \n [MWe]",
          "WH Temp [deg C]": "Producer\nTemp. [$\degree C$]",
          "Producer Wellhead Pressure [bar]": "Producer\nPressure [bar]",}

quantities = list(qdict.keys())
ylabels = list(qdict.values())

span = range(10, project.max_simulation_steps-1)
fig, axes = plot_cols({" ": project.df_records}, span, quantities, 
                         figsize=(10,12), ylabels=ylabels, legend_loc=False, dpi=100, 
                       formattime=False, return_figax=True)

### MATLAB: ULoopSBT_v2_2022_v18

In [ ]:
def baseload_strategy(project, m_prd=110):
    
    # UPDATE: we first update project state (also, we override parameters if needed)
    project.update_state(m_prd)
    
    # STEP: step power plant and reservoir
    T_inj = project.T_inj
    T_amb = project.T_amb
    m_turbine = project.m_g
    T_prd_wh = project.reservoir.T_prd_wh.mean()
    
    # step powerplant
    project.powerplant.step(m_turbine=m_turbine, 
                            T_prd_wh=T_prd_wh, 
                            T_amb=T_amb)
    # step reservoir
    project.reservoir.step(m_prd=m_prd,
                        T_inj=T_inj,
                        T_amb=T_amb)
    
    # RECORD: store the current project timestep
    project.record_step()

def generation_maximization_strategy(project, max_mass_flow=150):
    
    # UPDATE: we first update project state (also, we override parameters if needed)
    power_output_MWh_kg = project.powerplant.power_output_MWh_kg
    required_mass_flow_per_well = project.ppc / (power_output_MWh_kg * 3600 * project.num_prd + SMALL_NUM)
    m_prd = np.minimum(max_mass_flow, np.array(project.num_prd*[required_mass_flow_per_well])).astype(float)
    
    project.update_state(m_prd)
    
    # STEP: step power plant and reservoir
    T_inj = project.T_inj
    T_amb = project.T_amb
    m_turbine = project.m_g
    T_prd_wh = project.reservoir.T_prd_wh.mean()
    
    # step powerplant
    project.powerplant.step(m_turbine=m_turbine, 
                            T_prd_wh=T_prd_wh, 
                            T_amb=T_amb)
    # step reservoir
    project.reservoir.step(m_prd=m_prd,
                        T_inj=T_inj,
                        T_amb=T_amb)
    
    # RECORD: store the current project timestep
    project.record_step()
    
def revenue_maximization_strategy(project, m_prd=100, min_mass_flow=10, max_mass_flow=150):
    
    # UPDATE: we first update project state (also, we override parameters if needed)
    if "price_weight" not in project.df_market.columns:
        project.df_market["price_weight"] = project.df_market.price/project.df_market.price.rolling(24).mean().bfill()
        
    m_prd = np.clip(m_prd * project.df_market.loc[project.step_idx+1, "price_weight"], 
                    a_min=min_mass_flow, a_max=max_mass_flow)
    
    project.update_state(m_prd)
    
    # STEP: step power plant and reservoir
    T_inj = project.T_inj
    T_amb = project.T_amb
    m_turbine = project.m_g
    T_prd_wh = project.reservoir.T_prd_wh.mean()
    
    # step powerplant
    project.powerplant.step(m_turbine=m_turbine, 
                            T_prd_wh=T_prd_wh,
                            T_amb=T_amb)
    # step reservoir
    project.reservoir.step(m_prd=m_prd,
                        T_inj=T_inj,
                        T_amb=T_amb)
    
    # RECORD: store the current project timestep
    project.record_step()
    
def seasonal_strategy(project, m_prd=100, max_mass_flow=150):
    
    # UPDATE: we first update project state (also, we override parameters if needed)
    month = project.time_curr.month
    if (month >=5) and (month <=8):
        m_prd = max_mass_flow

    project.update_state(m_prd)
    
    # STEP: step power plant and reservoir
    T_inj = project.T_inj
    T_amb = project.T_amb
    m_turbine = project.m_g
    T_prd_wh = project.reservoir.T_prd_wh.mean()
    
    # step powerplant
    project.powerplant.step(m_turbine=m_turbine, 
                            T_prd_wh=T_prd_wh,
                            T_amb=T_amb)
    # step reservoir
    project.reservoir.step(m_prd=m_prd,
                        T_inj=T_inj,
                        T_amb=T_amb)
    
    # RECORD: store the current project timestep
    project.record_step()

In [ ]:
# further processing
cambium_scenario = "MidCase"
V_res = 18 #km2

T_inj = 20#60
surface_temp = 20 #deg C
num_prd = 1#2#1
m_well = 60#75#80 #kg/s
m_max = m_well * 2
m_min = 5 #m_well / 2

well_diam = 2 * 0.0762#0.1524
numberoflaterals = 12#4#12
lateral_length = 3500#20000#6500 # length from prd toe to inj toe
well_depth = 7000#7500 #18000/3.28 # meters
L = 30#10 #30
drilling_cost=1000 #$/meter
half_lateral_length = lateral_length/2
ppc = 10#500
state = "NM"
geothermal_gradient = 60 #(250-surface_temp)/(18000/3.28) #deg C/m based on Eavor publications
tres = surface_temp + geothermal_gradient * well_depth/1000
lateral_spacing = 100 #meters
dx = 500
reservoir_simulator_settings = {"on": False, "accuracy": 1, "DynamicFluidProperties": True}
closedloop_design = "eavor"
resample = "1w"

In [ ]:
# strategy = "baseload"
# strategy = "generation_maximization"
strategy = "revenue_maximization"
# strategy = "seasonal"


project_initialized, config = prepare_world(depth=well_depth, state=state,tres=tres, 
                                            lateral_spacing=lateral_spacing, lateral_length=lateral_length, 
                                            numberoflaterals=numberoflaterals, num_prd=num_prd, well_diam=well_diam, 
                                            m_well=m_well, drilling_cost=drilling_cost, L=L, ppc=ppc, 
                                            reservoir_simulator_settings=reservoir_simulator_settings, T_inj=T_inj, resample=resample, 
                                            closedloop_design=closedloop_design, dx=dx)

project = deepcopy(project_initialized)

for i in tqdm(range(project.max_simulation_steps-1)):
    days_passed = (project.time_curr - project.time_init).days
    if strategy == "baseload":
        baseload_strategy(project, m_well)
    elif strategy == "generation_maximization":
        generation_maximization_strategy(project, max_mass_flow=m_max)
    elif strategy == "revenue_maximization":
        revenue_maximization_strategy(project, m_prd=m_well, max_mass_flow=m_max, min_mass_flow=m_min)
    elif strategy == "seasonal":
        seasonal_strategy(project, m_prd=m_well, max_mass_flow=m_max)
        
    if  i % 365 == 0:
        print(project.time_curr)
        print(f"Year-{days_passed/365:0.1f}: {project.reservoir.Tres} deg C")
        
project.compute_economics()
NPV, ROI, PBP, IRR, PPA_NPV, PPA_ROI, PPA_PBP, PPA_IRR, LCOE, NET_GEN, df_annual = compute_npv(project.df_records, project.capex_total, project.opex_total,
                                                          project.baseline_year, project.L, project.d, ppa_price=75)

# print economics
print(f"LCOE: {LCOE:.0f} $/MWh")
print(f"NPV: {NPV:.0f} $MM")
print(f"ROI: {ROI:.1f} %")
print(f"PBP: {PBP:.0f} yrs")

# pickle.dump(projects, open("projects.pkl", "wb"))

In [ ]:
# fig = project.reservoir.plot_laterals()

In [ ]:
# include = ['Power Plant', 'Interconnection', 'Exploration', 
#            'Drilling', 'Injection Stimulation', 'Gathering System', 'Pumps', 'TES', 'Battery']
# costs = {k:v for k,v in project.present_capex_per_unit.items()}
# costs["Pumps"] = costs["Production Pumps"] + costs["Injection Pumps"]
# costs_final = {k:costs[k] for k in include}
# plot_ex({"": costs_final}, figsize=(7,7), dpi=100, fontsize=10)

In [ ]:
# include = ['Power Plant', 'Wellsite', 'Makeup Water']
# costs = {k:v for k,v in project.present_opex_per_unit.items()}
# # costs["Pumps"] = costs["Production Pumps"] + costs["Injection Pumps"]
# costs_final = {k:costs[k] for k in include}
# plot_ex({"": costs_final}, figsize=(7,7), dpi=100, fontsize=10)

In [ ]:
qdict = {
          "LMP [$/MWh]": "Electricity Price \n [$/MWh]",
          'M_Produced [kg/s]': "Field Mass \nFlow [kg/s]",
          "Atm Temp [deg C]": "Ambient Temp. \n [$\degree C$]",
#           "Res Temp [deg C]": "Reservoir Temp. \n [$\degree C$]",
#           "Pumping Power [MWe]": "Pumping Power \n [MWe]",
          "WH Temp [deg C]": "Producer\nTemp. [$\degree C$]",
#           "Producer Wellhead Pressure [bar]": "Producer\nPressure [bar]",
          'Inj Temp [deg C]': "Injector Temp. \n [$\degree C$]",
          "Net Power Output [MWe]": "Net Power \n [MWe]",
}

quantities = list(qdict.keys())
ylabels = list(qdict.values())

span = range(10, project.max_simulation_steps-1)
fig, axes = plot_cols_v2({" ": project.df_records}, span, quantities, 
                         figsize=(10,8), ylabels=ylabels, legend_loc=False, dpi=100, 
                       formattime=False, return_figax=True)

WEEKLY TIMESTEPPING:
- Baseload COE: 
140 $/MWh 
NPV: -72 $MM 
ROI: -64.1 % 
PBP: 0 yrs

- Generation Maximizing
LCOE: 154 $/MWh
NPV: -78 $MM
ROI: -69.8 %
PBP: 0 yrs

- Revenue Maximizing
LCOE: 149 $/MWh
NPV: -74 $MM
ROI: -66.0 %
PBP: 0 yrs

- Seasonal
LCOE: 147 $/MWh
NPV: -76 $MM
ROI: -67.5 %
PBP: 0 yrs

DAILY TIMESTEPPING:


- Revenue Maximizing
LCOE: 146 $/MWh
NPV: -73 $MM
ROI: -65.4 %
PBP: 0 yrs

### Other

In [ ]:
well_dpeth = 7000
lateral_length = 3000
numberoflaterals = 12
dx = lateral_length//10

In [ ]:
zinj = np.arange(0, -well_depth -well_depth/2, -well_depth/2).reshape(-1, 1)
yinj = np.zeros((len(zinj), 1))
xinj = -(lateral_length/2) * np.ones((len(zinj), 1))

# Coordinates of production well (coordinates are provided from bottom to top in the direction of flow)
zprod = np.arange(-well_depth, 0 + well_depth/2, well_depth/2).reshape(-1, 1)
yprod = np.zeros((len(zprod), 1))
xprod = (lateral_length/2) * np.ones((len(zprod), 1))

# Coordinates of laterals
xlat = np.repeat(np.arange(-lateral_length//2, lateral_length//2 + dx, dx)[:,None], numberoflaterals, axis=1)
if numberoflaterals > 1:
    lats = []
    for i in np.arange(numberoflaterals//2):
        arr = lateral_spacing * (i+1) * np.concatenate((np.cos(np.linspace(-np.pi/2, 0, 3)), np.ones(xlat.shape[0]-6), \
                            np.cos(np.linspace(0, np.pi/2, 3))))
        lats.extend([arr, -arr])
    # if odd number of laterals, then include a center lateral
    if numberoflaterals % 2 == 1:
        lats.append(np.zeros_like(arr))

    ylat = np.array(lats).T
else:
    ylat = np.zeros_like(xlat)
zlat = -well_depth * np.ones_like(xlat)

# Merge x-, y-, and z-coordinates
x = np.concatenate((xinj, xprod, xlat.flatten(order="F")[:,None]))
y = np.concatenate((yinj, yprod, ylat.flatten(order="F")[:,None]))
z = np.concatenate((zinj, zprod, zlat.flatten(order="F")[:,None]))

In [ ]:
well_depth = 7000
vertical_section_length = 2000
vertical_well_spacing = 100
junction_depth = 4000
angle = 20*np.pi/180;
element_length = 150
numberoflaterals = 2

In [ ]:
N = 5
# generate inj well profile
zinj = np.linspace(0, -vertical_section_length, N).reshape(-1, 1)
yinj = np.zeros((len(zinj), 1))
xinj = np.zeros((len(zinj), 1))

inclined_length = abs(-junction_depth - zinj[-1])/np.cos(angle)

# zinj_inclined_length = np.arange(zinj[-1]-step_size, -junction_depth-step_size , -step_size).reshape(-1, 1)
zinj_inclined_length  = np.linspace(np.round((zinj[-1] - junction_depth)/2), -junction_depth, N)
yinj_inclined_length = np.zeros((len(zinj_inclined_length), 1))
xend = xinj[-1]+inclined_length * np.sin(angle)
xinj_inclined_length = np.linspace((xinj[-1]+xend)/2, xend, N)

zinj = np.concatenate((zinj, zinj_inclined_length))
xinj = np.concatenate((xinj, xinj_inclined_length))
yinj = np.concatenate((yinj, yinj_inclined_length))

# generate prod well profile
zprod = np.flip(zinj);
xprod = np.flip(xinj);
yprod = np.flip(yinj) + vertical_well_spacing;

# Generate Laterals
# Injection points
x_ip = np.zeros((1,numberoflaterals))
y_ip = np.zeros((1,numberoflaterals))
z_ip = np.zeros((1,numberoflaterals))
for i in range(numberoflaterals):
    y_ip[0, i] = yinj[-1]-(lateral_spacing*(numberoflaterals-1))/2+i*lateral_spacing-(yinj[-1]-yprod[-1])/2;
    x_ip[0, i] = xinj[-1]+element_length*3*np.sin(angle);
    z_ip[0, i] = zinj[-1]-element_length*3*np.cos(angle);

# Lateral feedways
x_feed = np.zeros((N, numberoflaterals))
y_feed = np.zeros((N, numberoflaterals))
z_feed = np.zeros((N, numberoflaterals))
for i in range(numberoflaterals):
#     x_feed[:,i] = np.linspace(3/4 * xinj[-1, 0] + x_ip[0, i]/4,xinj[-1, 0]/4 + x_ip[0, i]*3/4,N)
#     y_feed[:,i] = np.linspace(3/4 * yinj[-1, 0]+y_ip[0, i]/4, yinj[-1, 0]/4+y_ip[0, i]*3/4,N)
#     z_feed[:,i] = np.linspace(3/4 * zinj[-1, 0]+z_ip[0, 0/4, zinj[-1, 0]/4+z_ip[0, i]*3/4,N)
    # we space things out by 1% to avoid zero segment lengths in SBT
    x_feed[:,i] = np.linspace(xinj[-1, 0], x_ip[0, i] * 0.99, N)
    y_feed[:,i] = np.linspace(yinj[-1, 0], y_ip[0, i] * 0.99, N)
    z_feed[:,i] = np.linspace(zinj[-1, 0] , z_ip[0, i] * 0.99, N)

    
# lateral template ...
lateral_length = (well_depth-abs(z_ip[0, -1]))/np.cos(angle)
z_template_lateral = np.linspace(z_ip[0, -1], -well_depth, N)
z_template_lateral = np.concatenate((z_template_lateral, 
                                     z_template_lateral[[-1]]+element_length,
                                     z_template_lateral[::-1]+2*element_length
                                    ))
z_template_lateral = np.repeat(z_template_lateral[None], numberoflaterals, axis=0).T

xend = x_ip[0, -1]+lateral_length * np.sin(angle)
x_template_lateral = np.concatenate((np.linspace(x_ip[0, -1], xend, N),
                                     np.array([xend]),
                                     np.linspace(x_ip[0, -1], xend, N)[::-1]
                                   ))
x_template_lateral = np.repeat(x_template_lateral[None], numberoflaterals, axis=0).T

y_template_lateral = np.repeat(y_ip, x_template_lateral.shape[0], axis=0)

# Lateral returns
x_return = np.zeros((N, numberoflaterals))
y_return = np.zeros((N, numberoflaterals))
z_return = np.zeros((N, numberoflaterals))
for i in range(numberoflaterals):
#     x_return[:,i] = np.linspace(3/4 * xprod[0, 0] + x_template_lateral[0, i]/4, xprod[0, 0]/4 + x_template_lateral[0, i]*3/4,N)
#     y_return[:,i] = np.linspace(3/4 * yprod[0, 0]+y_template_lateral[0, i]/4, yprod[0, 0]/4+y_template_lateral[0, i]*3/4,N)
#     z_return[:,i] = np.linspace(3/4 * zprod[0, 0]+z_template_lateral[0, i]/4, zprod[0, 0]/4+z_template_lateral[0, i]*3/4,N)
    x_return[:,i] = np.linspace(x_template_lateral[-1, i] * 1.01, xprod[0, 0],N)
    y_return[:,i] = np.linspace(y_template_lateral[-1, i] * 1.01, yprod[0, 0],N)
    z_return[:,i] = np.linspace(z_template_lateral[-1, i] * 1.01, zprod[0, 0],N)
    
zlat = np.vstack((z_feed, z_template_lateral, z_return))
xlat = np.vstack((x_feed, x_template_lateral, x_return))
ylat = np.vstack((y_feed, y_template_lateral, y_return))

x = np.concatenate((xinj, xprod, xlat.flatten(order="F")[:,None]))
y = np.concatenate((yinj, yprod, ylat.flatten(order="F")[:,None]))
z = np.concatenate((zinj, zprod, zlat.flatten(order="F")[:,None]))
Deltaz = np.sqrt((x[1:] - x[:-1]) ** 2 + (y[1:] - y[:-1]) ** 2 + (z[1:] - z[:-1]) ** 2)



In [ ]:
np.where(Deltaz ==0)

In [ ]:
fig = plt.figure(dpi=150)
ax = fig.add_subplot(111, projection='3d')

ax.plot(xinj, yinj, zinj, 'tab:blue', linewidth=4)
ax.plot(xprod, yprod, zprod, 'tab:red', linewidth=4)
for j in range(xlat.shape[1]):
    ax.plot(xlat[:,j], ylat[:,j], zlat[:,j],
            linewidth=2, color="black")#, label=f"Lateral-{j}")

ax.set_xlim([np.min(xlat) - 1000, np.max(xlat) + 200])
ax.set_ylim([np.min(ylat) - 1000, np.max(ylat) + 200])
ax.set_zlim([np.min(zlat) - 1000, 0])
plt.legend(["Injector", "Producer", "Laterals"])

In [ ]:
def solution(S):
    # Convert the input string to an integer
    N = int(S)
    # Convert the integer to hexadecimal representation
    hex_string = hex(N)[2:].upper()  # remove the '0x' prefix and convert to uppercase

    # Create a dictionary to translate 0 to O and 1 to I
    translation = str.maketrans('01', 'OI')
    hexspeak_string = hex_string.translate(translation)

    # Check if the hexspeak_string contains only valid Hexspeak characters
    for char in hexspeak_string:
        if char not in 'ABCDEFIO':
            return "ERROR"

    return hexspeak_string

# Example usage:
print(solution("257"))  # Expected output: "IOI"
print(solution("3"))  